## 8. 객체 참조, 가변성, 재활용
- 파이썬의 변수
- 객체의 정체성, 동질성, 별명
- 얕은 복사와 깊은 복사
- 참조 및 함수 매개변수
- 가비지 컬렉션, del



### 8.1. 변수는 상자가 아니다

파이썬 변수는 자바에서의 참조 변수와 같으므로 변수는 객체에 붙은 레이블이라고 생각하는 것이 좋다. 

In [1]:
a = [1, 2, 3]
b = a # a와 b가 같은 참조를 가지고 있기 때문에 
a.append(4) # a를 변경하면 
print(b) # b도 같이 변경된다.

[1, 2, 3, 4]


참조 변수의 경우 변수가 객체에 할당되었다는 표현이 객체를 변수에 할당했다는 표현보다 훨씬 타당하다. 결국 객체는 변수가 할당되기 전에 생성된다.

In [4]:
class Gizmo:
  def __init__(self):
    print("Gizmo id: %d" % id(self))
    
x = Gizmo()
print(x)

'''
Gizmo id: 139991375373144

---------------------------------------------------------------------------
TypeError                                 Traceback (most recent call last)
<ipython-input-4-747feaee16b0> in <module>()
      5 x = Gizmo()
      6 print(x)
----> 7 y = Gizmo()*10 # Gizmo 객체에 숫자를 곱하면 예외가 발생한다.
      8 print(y)
      9 

TypeError: unsupported operand type(s) for *: 'Gizmo' and 'int'
'''
# 객체가 생성이 먼저 되고 숫자와의 곱을 수행

y = Gizmo()*10 # Gizmo 객체에 숫자를 곱하면 예외가 발생한다.
print(y)

print(dir()) # y 생성하다가 에러났기 때문에 y는 생성되지 않음

Gizmo id: 139991375372528
Gizmo id: 139991375373144


TypeError: ignored

### 8.2. 정체성, 동질성, 별명

변수는 단지 레이블일 뿐이므로 객체에 여러 레이블을 붙일 수 있다. 여러 레이블을 붙이는 것을 별명이라고 한다. 

In [5]:
charles = {'name' : 'Charles L. Dodgson', 'born' : 1832}
lewis = charles

print(lewis is charles) # is는 ID가 같은지 비교
print(id(lewis), id(charles))

lewis['balance'] = 950

print(charles)

True
139991375448464 139991375448464
{'name': 'Charles L. Dodgson', 'born': 1832, 'balance': 950}


In [7]:
alex = {'name': 'Charles L. Dodgson', 'born': 1832, 'balance': 950}

print(alex == charles) # dict 클래스에서 __eq__() 호출 : 값이 같은지 비교
print(alex is not charles)
print(id(alex), id(charles)) # id 다름

True
True
139991375342400 139991375448464


>> **모든 객체는 정체성, 자료형, 값을 가지고 있다. 객체의 정체성은 일단 생성한 후에는 결코 변경되지 않는다. 정체성은 메모리 내의 객체 주소라고 생각할 수 있다. is 연산자는 두 객체의 정체성을 비교한다. id() 함수는 정체성을 나타내는 정수를 반환한다.**


#### 8.2.1. == 연산자와 is 연산자 간의 선택
== 연산자가 객체의 값을 비교하는 반면, is 연산자는 객체의 정체성을 비교한다. 정체성보다 값을 비교하는 경우가 많으므로, == 를 자주 사용한다. 하지만 변수를 싱글턴과 비교할 때는 is 연산자를 사용해야 한다. is 연산자를 사용할 때는 변수를 None과 비교하는 경우가 많다. 

#### 8.2.2. 튜플의 상대적 불변성
튜플은 객체에 대한 참조를 담는다. 참조된 항목이 가변형이면 튜플 자체는 불변형이지만 참조된 항목은 변할 수 있다. 튜플의 불변성은 tuple 데이터 구조체의 물리적인 내용, 참조 자체 만을 말하는 것이지, 참조된 객체까지 불변성을 가지는 것은 아니다. 

In [8]:
t1 = (1, 2, [30, 40])
t2 = (1, 2, [30, 40])
print(t1 == t2)
print(id(t1[-1]))
t1[-1].append(99)
print(t1)

print(id(t1[-1])) # id는 바뀌지 않았지만
print(t1 == t2) # 값이 바뀜


True
139991375807432
(1, 2, [30, 40, 99])
139991375807432
False


### 8.3. 기본 복사는 얕은 복사

리스트나 대부분의 내장 가변 컬렉션을 복사하는 가장 쉬운 방법은 자료형 자체의 내장 생성자를 사용하는 것이다. 

In [10]:
l1 = [3, [55,44], (7,8,9)]
l2 = list(l1) # 얕은 사본
print(l2)
print(l1 == l2)
print(l2 is l1)
print()

print(id(l1[-1]), id(l2[-1])) # 같은 튜플을 참조함

[3, [55, 44], (7, 8, 9)]
True
False

139991375796840 139991375796840


생성자나 \[:\]를 사용하면 얕은 사본을 생성한다. 최상위 컨테이너는 복제하지만 사본은 원래 컨테이너에 들어 있던 동일 객체에 대한 참조로 채워진다. 따라서, 가변 컨테이너를 얕은 복사를 할 경우 문제가 생길 수 있다. 

In [12]:
l1 = [3, [55,44], (7,8,9)]
l2 = list(l1)
l1.append(100)
l1[1].remove(55)
print("l1 : ", l1)
print("l2 : ", l2) # l1, l2에서 모두 55가 사라짐

l2[1] += [33, 22]
l2[2] += (10, 11) # 새로운 튜플을 만들어서 다시 바인딩
print("l1 : ", l1)
print("l2 : ", l2)

print(id(l1[-1]), id(l2[-1])) # 튜플 ID가 달라짐

l1 :  [3, [44], (7, 8, 9), 100]
l2 :  [3, [44], (7, 8, 9)]
l1 :  [3, [44, 33, 22], (7, 8, 9), 100]
l2 :  [3, [44, 33, 22], (7, 8, 9, 10, 11)]
10938656 139991376042208


#### 8.3.1. 객체의 깊은 복사와 얕은 복사

얕은 복사로 늘 문제가 생기는 것은 아니지만, 내포된 객체의 참조를 공유하지 않도록 깊게 복사할 필요가 있는 경우가 있다. copy 모듈이 제공하는 deepcopy() 함수는 깊은 복사를, copy()는 얕은 복사를 지원한다.

In [0]:
class Bus:
  def __init__(self, passengers=None):
    if passengers is None:
      self.passengers = []
    else :
      self.passengers = list(passengers)
      
    print(id(self.passengers)," : ", self.passengers)
      
      
  def pick(self, name):
    self.passengers.append(name)
    print(self.passengers)
    
    
  def drop(self, name) :
    self.passengers.remove(name)
    print(self.passengers)
 

In [17]:
import copy
bus1 = Bus(["Alice", "Bill", "Claire", "David"])
bus2 = copy.copy(bus1)

bus3 = copy.deepcopy(bus1)

print(id(bus1.passengers), id(bus2.passengers), id(bus3.passengers)) # bus1.passengers is bus2.passengers

bus1.drop("Bill")
print(bus2.passengers)
print(bus3.passengers) # Bus3에는 영향 없음

139991375456456  :  ['Alice', 'Bill', 'Claire', 'David']
139991375456456 139991375456456 139991375454280
['Alice', 'Claire', 'David']
['Alice', 'Claire', 'David']
['Alice', 'Bill', 'Claire', 'David']


일반적으로 깊은 사본을 만드는 일은 간단치 않다. 객체 안에 순환 참조가 있으면 단순한 알고리즘은 무한 루프에 빠질 수 있다. deepcopy()는 순환 참조를 제대로 처리하기 위해 이미 복사한 객체에 대한 참조를 기억하고 있다. 

In [19]:
a = [10, 20]
b = [a, 30]

a.append(b)
print(a)

from copy import deepcopy
c = deepcopy(a)
print(c)

[10, 20, [[...], 30]]
[10, 20, [[...], 30]]


### 8.4. 참조로서의 함수 매개변수

파이썬은 공유로 호출(Call by Sharing) 하는 매개변수 전달 방식만 지원한다. 공유로 호출한다는 말은 함수의 각 매개변수가 인수로 전달받은 각 참조의 사본을 받는다는 것이다. 이런 체계의 결과로, 함수는 인수로 전달받은 모든 가변 객체를 변경할 수 있지만, 객체의 정체성 자체는 변경할 수 없다. 

In [21]:
def f(a, b):
  a += b
  return a


x = 1
y = 2

print("f(x, y) : ", f(x, y))
print("(x, y) : ", x, y)

a = [1, 2]
b = [3, 4]
print("f(a, b) : ", f(a, b))
print("(a, b) : ", a, b)

t = (10, 20)
u = (30, 40)
print("f(t, u) : ", f(t, u))
print("(t, u) : ", t, u)


f(x, y) :  3
(x, y) :  1 2
f(a, b) :  [1, 2, 3, 4]
(a, b) :  [1, 2, 3, 4] [3, 4]
f(t, u) :  (10, 20, 30, 40)
(t, u) :  (10, 20) (30, 40)


#### 8.4.1. 가변형을 매개변수 기본값으로 사용하기 : 좋지 않은 생각

기본값을 가진 선택적 인수는 파이썬 함수 정의에서 아주 좋은 기능이지만, 매개변수 기본값으로 가변 객체를 사용하는 것은 피해야 한다. 

In [0]:
class HauntedBus:
  def __init__(self, passengers=[]): # 빈리스트가 class 정의될 때 만들어지나?
    self.passengers = passengers
    print(id(self.passengers)," : ", self.passengers)
      
      
  def pick(self, name):
    self.passengers.append(name)
    print(self.passengers)
    
    
  def drop(self, name) :
    self.passengers.remove(name)
    print(self.passengers)
 

In [25]:
bus1 = HauntedBus(["Alice", "Bill"])
print(bus1.passengers)
bus1.pick("Charles")
bus1.drop("Alice")
print(bus1.passengers)
print()

bus2 = HauntedBus()
bus2.pick("Carrie")
print(bus2.passengers)
print()

bus3 = HauntedBus()
print(bus3.passengers) # 기본값이 비어있지 않다!
bus3.pick("Dave")
print(bus2.passengers)

print(bus2.passengers is bus3.passengers) # bus2와 bus3이 동일한 리스트를 참조
print(bus1.passengers)


139991375357192  :  ['Alice', 'Bill']
['Alice', 'Bill']
['Alice', 'Bill', 'Charles']
['Bill', 'Charles']
['Bill', 'Charles']

139991375807496  :  []
['Carrie']
['Carrie']

139991375807496  :  ['Carrie']
['Carrie']
['Carrie', 'Dave']
['Carrie', 'Dave']
True
['Bill', 'Charles']


In [29]:
dir(HauntedBus.__init__)
print(HauntedBus.__init__.__defaults__)
print(HauntedBus.__init__.__defaults__[0] is bus2.passengers) # 기본값이 bus2.passengers에 바인딩

(['Carrie', 'Dave'],)
True


#### 8.4.2. 가변 매개변수에 대한 방어적 프로그래밍

가변 매개변수를 받는 함수를 구현할 때는 전달된 인수가 변경될 것이라는 것을 호출자가 예상할 수 있는지 없는지 신중하게 고려해야 한다. 

In [0]:
class TwilightBus:
  def __init__(self, passengers=None):
    if passengers is None:
      self.passengers = []
    else :
      self.passengers = passengers # 리스트를 새로 만들지 않고 원래의 리스트 참조를 그대로 사용 -> list(passengerse)
      
      
  def pick(self, name):
    self.passengers.append(name)
    print(self.passengers)
    
    
  def drop(self, name) :
    self.passengers.remove(name)
    print(self.passengers)
 

In [33]:
basketball_team = ['Sue', 'Tina', 'Maya', 'Diana', 'Pat']
bus = TwilightBus(basketball_team)
bus.drop("Tina")
bus.drop("Pat")
basketball_team # 원래 리스트가 변경되었음

['Sue', 'Maya', 'Diana', 'Pat']
['Sue', 'Maya', 'Diana']


['Sue', 'Maya', 'Diana']

### 8.5. del과 가비지 컬렉션

**del 명령은 이름을 제거하는 것이지, 객체를 제거하는 것이 아니다.** del 명령의 결과로 객체가 가비지 컬렉트될 수 있지만, 제거된 변수가 객체를 참조하는 최후의 변수거나 객체에 도달할 수 없을 때에만 가비지 컬렉트된다.  

본질적으로 각 객체는 얼마나 많은 참조가 자신을 가리키는지 개수(refcount)를 세고 있다. refcount가 0이 되자마자 CPython이 객체의 \_\_del\_\_() 메서드를 호출하고 객체에 할당되어 있는 메모리를 해제함으로써 객체가 제거된다.

In [34]:
import weakref

s1 = {1, 2, 3}
s2 = s1
def bye():
  print("Gone with the wind...")
  
ender = weakref.finalize(s1, bye)
print(ender.alive)

del s1 # 객체를 없앤 것이 아니라 refcount -1 하는 것
print(ender.alive)
s2 = 'spam' # {1, 2, 3} 리스트 참조를 없애자마자 사라짐
print(ender.alive)


True
True
Gone with the wind...
False


### 8.6. 약한 참조

객체가 메모리에 유지되거나 유지되지 않도록 만드는 것은 참조의 존재 여부다. 객체 참조 카운트가 0이 되면 가비지 컬렉터는 해당 객체를 제거한다. 그러나 불필요하게 객체를 유지시키지 않으면서 객체를 참조할 수 있으면 도움이 되는 경우가 종종있는데, 캐시가 대표적인 경우이다. 

약한 참조는 참조 카운트를 증가시키지 않고 객체를 참조한다. 


In [35]:
import weakref

a_set = {0, 1}
wref = weakref.ref(a_set) # 참조된 객체를 반환
print(wref)
print(wref())
a_set = {2, 3, 4}
print(wref())

print(wref() is None)

<weakref at 0x7f52482e0598; to 'set' at 0x7f5248410f28>
{0, 1}
None
True
True


#### 8.6.1. WeakValueDictionary 촌극

WeakValueDictionary 클래스는 객체에 대한 약한 참조를 값으로 가지는 가변 매핑을 구현한다. 참조된 객체가 프로그램 다른 곳에서 가비지 컬렉트되면 해당 키도 WeakValueDictionary에서 자동으로 제거된다.

In [0]:
class Cheese:
  def __init__(self, kind):
    self.kind = kind
    
  def __repr__(self):
    return "Cheese(%r)" % self.kind

In [37]:
import weakref

stock = weakref.WeakValueDictionary()
catalog = [Cheese('Red Leicester'), Cheese('Tilsit'), Cheese('Brie'), Cheese('Parmesan')]

for cheese in catalog:
  stock[cheese.kind] = cheese
  
  
print(sorted(stock.keys()))
del catalog

print(sorted(stock.keys())) # Parmesan이 남아있다? -> for loop cheese 변수에 남아있어서
del cheese

print(sorted(stock.keys()))

['Brie', 'Parmesan', 'Red Leicester', 'Tilsit']
['Parmesan']
[]


#### 8.6.2. 약한 참조의 한계

모든 파이썬 객체가 약한 참조의 대상이 될 수 있는 것은 아니다. 기본적인 list와 dict 객체는 참조 대상이 될 수 없지만, 이 클래스들의 서브 클래스는 이 문제를 다음처럼 쉽게 고칠 수 있다.

In [38]:
t1 = (1, 2, 3)
t2 = tuple(t1)

print(t1 is t2)
t3 = t1[:]
print(t1 is t3) # 참조가 반환됨

True
True


In [39]:
t1 = (1, 2, 3)
t3 = (1, 2, 3) # 튜플은 새로 만든다
print(t1 is t3)

s1 = "ABC"
s2 = "ABC"
print(s1 is s2) # 동일한 문자열을 참조

False
True
